In [2]:
import pandas as pd
import numpy as np

train_data = pd.read_csv("train.tsv", delimiter = '\t', encoding = 'UTF-8')
train_data.groupby('label').count()
train_data.query('label == 1').head(3)

,comment,label
8,אדוני הנשיא סליחה על הבוטות תפסיק לזיין את השכ...,1
10,ואני עוד חשבתי שהגיע נשיא נורמלי,1
11,נשיא המדינה הלבן והמנותק חכם בדיבורים . הוא לא...,1


In [ ]:
train_data = train_data[(train_data['label'] == 1) | (train_data['label'] == 0)]
train_data.groupby('label').count()

,comment
label,
0,4017
1,1744


In [ ]:
test_data = pd.read_csv("token_test.tsv", delimiter = '\t', encoding = 'UTF-8')
test_data.columns = ['comment','label']

test_data = test_data[(test_data['label'] == 1) | (test_data['label'] == 0)]
test_data.groupby('label').count()

,comment
label,
0,1697
1,790


In [ ]:
from imblearn.under_sampling import RandomUnderSampler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier

X_train = train_data['comment']
y_train = train_data['label']
vectorizer = CountVectorizer(analyzer='char', ngram_range=(1, 15), min_df = 50)
X_train = vectorizer.fit_transform(X_train)
rus = RandomUnderSampler(random_state = 42)
X_train, y_train = rus.fit_resample(X_train, y_train)

print(f"number of features: {X_train.shape[1]}, and {X_train.shape[0]} samples")

number of features: 7564, and 3488 samples


In [ ]:
X_test = test_data['comment']
y_test = test_data['label']
X_test = vectorizer.transform(X_test)
X_test, y_test = rus.fit_resample(X_test, y_test)

In [ ]:
logreg = LogisticRegression(max_iter = 3000, C = 0.2, penalty = "l2")
logreg.fit(X_train, y_train)

print(f"Train score: {logreg.score(X_train, y_train):.1%}")
print(f"Test score: {logreg.score(X_test, y_test):.1%}")

Train score: 99.2%
Test score: 89.7%


In [ ]:
rf = RandomForestClassifier(min_samples_leaf =5)
rf.fit(X_train, y_train)
print(f"Train score: {rf.score(X_train, y_train):.1%}")
print(f"Test score: {rf.score(X_test, y_test):.1%}")

Train score: 91.5%
Test score: 83.9%


In [ ]:
mlp = MLPClassifier(hidden_layer_sizes=(64,32,16,8), activation = 'relu', alpha=0.01, batch_size='auto', max_iter=200)
mlp.fit(X_train, y_train)

print(f"Train score: {mlp.score(X_train, y_train):.1%}")
print(f"Test score: {mlp.score(X_test, y_test):.1%}")

Train score: 100.0%
Test score: 90.1%


In [ ]:
text = "אתה ראש ממשלה גרוע תעוף לנו מהעיניים "
print(text)
X = np.array([text])
X = vectorizer.transform(X)
prediction = logreg.predict(X)
if prediction:
  print("- סנטימנט שלילי - \n")
else:
  print("+ נייטראלי + \n")

text = "מזל טוב ! בהחלט היה מגיע לך איש יקר ושיהיה לך בהצלחה !!!!"
print(text)
X = np.array([text])
X = vectorizer.transform(X)
prediction = logreg.predict(X)
if prediction:
  print("- סנטימנט שלילי - \n")
else:
  print("+ נייטראלי / חיובי+ \n")

אתה ראש ממשלה גרוע תעוף לנו מהעיניים 
- סנטימנט שלילי - 

אנחנו אוהבים אותך!!! בהצלחה!
+ נייטראלי / חיובי+ 



In [ ]:
text = "אתה ראש ממשלה גרוע תעוף לנו מהעיניים "
print(text)
X = np.array([text])
X = vectorizer.transform(X)
prediction = logreg.predict(X)
if prediction:
  print("- סנטימנט שלילי - \n")
else:
  print("+ נייטראלי + \n")

text = "אנחנו אוהבים אותך!!! בהצלחה!"
print(text)
X = np.array([text])
X = vectorizer.transform(X)
prediction = logreg.predict(X)
if prediction:
  print("- סנטימנט שלילי - \n")
else:
  print("+ נייטראלי / חיובי+ \n")

In [ ]:
text_input = ""
print("ליציאה הכנס 0 \n")
while text_input != '0':
  print("הכנס טקסט לבדיקה: \n")
  text_input = input()
  if text_input != '0':
    X = np.array([text_input])
    X = vectorizer.transform(X)
    prediction = mlp.predict(X)
    if prediction:
      print("- סנטימנט שלילי - \n")
    else:
      print("+ נייטראלי / חיובי+ \n")
  else:
    print("\n!להתראות")

ליציאה הכנס 0 

הכנס טקסט לבדיקה: 

זוהרה כולנו כהנא ! : )
- סנטימנט שלילי - 

הכנס טקסט לבדיקה: 

כהנא צדק
- סנטימנט שלילי - 

הכנס טקסט לבדיקה: 

מתבייש בך כבוד הנשיא ....
+ נייטראלי / חיובי+ 

הכנס טקסט לבדיקה: 

 לא נשיא של מדינה יהודית .
- סנטימנט שלילי - 

הכנס טקסט לבדיקה: 

